In [ ]:
!pip install -q bitsandbytes accelerate peft
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.3 M

In [ ]:
from huggingface_hub import login
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)


In [ ]:
import torch
torch.cuda.empty_cache()  # Clear unused GPU memory
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from datasets import load_dataset
dataset = load_dataset("ammarnasr/the-stack-java-clean", split='train', streaming=True)
print(dataset)

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

IterableDataset({
    features: ['hexsha', 'size', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction'],
    num_shards: 8
})


In [ ]:
# Collect code samples
code_samples = []
for i, sample in enumerate(dataset):
    code_samples.append(sample['content'])
    if i == 15000:
        break

for idx, code in enumerate(code_samples):
    print(f"Code Sample {idx + 1}:\n{code}\n---")

Buffered data was truncated after reaching the output size limit.

In [ ]:
training_data = [{"prompt": code, "completion": code} for code in code_samples]

In [ ]:
print(f"Training data size: {len(training_data)}")

Training data size: 15001


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

text = training_data[0]['prompt']
inputs = tokenizer(
    text,
    max_length=512,  # Reduce sequence length
    truncation=True,
    return_tensors="pt",
    add_special_tokens=True, # Add special tokens
)
inputs = {k: v.to(device) for k, v in inputs.items()}

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    ),
    device_map="auto"
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)

In [ ]:
model.eval()

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): 

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4bit(in_f

In [ ]:
model.gradient_checkpointing_enable()
model = model.half().to(device)

In [ ]:
inputs = tokenizer(text, max_length=512, truncation=True, return_tensors="pt")
inputs = {
    k: v.to(device) if v.dtype == torch.float else v.to(device)
    for k, v in inputs.items()
}
 # Move + convert to fp16

In [ ]:
model.train()
labels = inputs['input_ids'].clone()
outputs = model(**inputs, labels=labels)
loss = outputs.loss
print(f"Sample loss: {loss.item()}")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Sample loss: 1.138396143913269


In [ ]:
model.print_trainable_parameters()

trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


In [ ]:
test_prompt = """ class DynamicProgramming {
    static int knapsack(int[] wt, int[] val, int n, int W) {
        int dp[][] = new int[n][W + 1];

        for (int i = wt[0]; i <= W; i++) {
            dp[0][i] = val[0];
        }

        for (int ind = 1; ind < n; ind++) {
            for (int cap = 0; cap <= W; cap++) {"""


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id


In [ ]:
test_inputs = tokenizer(
    test_prompt,
    return_tensors="pt",
    truncation=True,
    padding=True
).to(model.device)

In [ ]:
print(test_inputs['input_ids'].max(), tokenizer.vocab_size)

tensor(50284, device='cuda:0') 50257


In [ ]:
generated_ids = model.generate(
    input_ids=test_inputs['input_ids'],
    attention_mask=test_inputs['attention_mask'],
    max_new_tokens=450,
    do_sample=True,
    temperature=0.3,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)


 class DynamicProgramming {
    static int knapsack(int[] wt, int[] val, int n, int W) {
        int dp[][] = new int[n][W + 1];
        
        for (int i = wt[0]; i <= W; i++) {
            dp[0][i] = val[0];
        }
        
        for (int ind = 1; ind < n; ind++) {
            for (int cap = 0; cap <= W; cap++) {
                if (cap >= wt[ind]) {
                    dp[ind][cap] = Math.max(val[ind] + dp[ind - 1][cap - wt[ind]], dp[ind - 1][cap]);
                } else {
                    dp[ind][cap] = dp[ind - 1][cap];
                }
            }
        }
        
        return dp[n - 1][W];
    }
    
    public static void main(String[] args) {
        int n = 4;
        int[] wt = {4, 3, 2, 2};
        int[] val = {2, 3, 5, 7};
        System.out.println(knapsack(wt, val, n, 5));
    }
}

A:

First, there is a bug in your code:
dp[ind][cap] = Math.max(val[ind] + dp[ind - 1][cap - wt[ind]], dp[ind - 1][cap]);

For some values of ind and cap, val[ind] is 0 (or n